# Applied Data Science Capstone Week 3
## Create the dataframe from wikipedia

#### Import packages that are going to be used

In [2]:
import numpy as np
import pandas as pd
import requests # Library for web scraping
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

#### Web scraping from wikipedia

In [5]:
ignore = ssl.create_default_context()
ignore.check_hostname = False
ignore.verify_mode = ssl.CERT_NONE
wikipedia_page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wikipedia_page, 'lxml')

#### Build a table to contain the table on the website

In [21]:
table = soup.find('table')
rows = table.find_all('tr')
data = []
for row in rows:
    data.append([t.text.strip() for t in row.find_all('td')])

#### Create data frames with removing and droping unnessesary rows and NAs and replacing / to ,

In [23]:
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]
df.drop(df[df['Borough']=='Not assigned'].index,axis=0, inplace=True)
df1 = df.reset_index()
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))
df2.loc[df2['Neighbourhood']=='Not assigned','Neighbourhood']=df2.loc[df2['Neighbourhood']=='Not assigned','Borough']
df3 = df2.reset_index()
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")
df3['Neighbourhood'] = df3['Neighbourhood'].str.replace('/',',')
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Add coordinates to the data frame

#### Load csv file and print to check

In [25]:
coordinates = pd.read_csv('Geospatial_Coordinates.csv')
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
df4 = df3.set_index('PostalCode')
temp = coordinates.set_index('Postal Code')
toronto_df = pd.concat([df4, temp], axis=1, join='inner')

toronto_df.index.name = 'PostalCode'
toronto_df.reset_index(inplace=True)

print(toronto_df.shape)
toronto_df.head()

(103, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
